## Imports

In [22]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os

In [23]:
df = pd.read_csv('ROSS_map_template_popup_pin - Sheet1.csv')

## Image URL collector

In [24]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [25]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

df['image_links'] = links

In [26]:
len(links)

10

## Article link formatter

In [27]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2022/11...
1    <a href="https://therealdeal.com/miami/2021/09...
2    <a href="https://therealdeal.com/miami/2022/10...
3    <a href="https://therealdeal.com/miami/2022/09...
4    <a href="https://therealdeal.com/miami/2021/01...
5    <a href="https://therealdeal.com/miami/2021/05...
6    <a href="https://therealdeal.com/miami/2021/06...
7    <a href="https://therealdeal.com/miami/2022/02...
8    <a href="https://therealdeal.com/miami/2022/08...
9    <a href="https://therealdeal.com/miami/2022/09...
Name: description_link, dtype: object

## Google Maps API Geolocater Setup

In [28]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [29]:
df['geo_address'] = df['full_address'] + df['county']
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [30]:
df.columns

Index(['full_address', 'county', 'buy_price', 'sell_price', 'condos', 'apts',
       'office_sqft', 'description', 'story_link', 'additional_links',
       'other_notes', 'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [31]:
df

,full_address,county,buy_price,sell_price,condos,apts,office_sqft,description,story_link,additional_links,...,image_links,short_description,remaining_desc,description_link,geo_address,loc,point,lat,lon,altitude
0,"2601 Solano Avenue, Cooper City",Broward,55500000,NaN,NaN,300.0,NaN,"In November, Steve Ross' Related paid $55.5 mi...",https://therealdeal.com/miami/2022/11/15/steve...,NaN,...,https://s11.therealdeal.com/trd/m/up/2022/11/S...,"In November, Steve",Ross' Related paid $55.5 million for 300 affo...,"<a href=""https://therealdeal.com/miami/2022/11...","2601 Solano Avenue, Cooper CityBroward","(2601 Solano Ave, Hollywood, FL 33024, USA, (2...","(26.0322042, -80.25221359999999, 0.0)",26.032204,-80.252214,0.0
1,208 Fern Street and 400 South Olive Avenue,West Palm Beach,65000000,NaN,NaN,330.0,NaN,"Last September, Ross paid a total of $65 milli...",https://therealdeal.com/miami/2021/09/03/relat...,NaN,...,https://s12.therealdeal.com/trd/m/up/2021/09/m...,"Last September, Ross",paid a total of $65 million for two affordabl...,"<a href=""https://therealdeal.com/miami/2021/09...",208 Fern Street and 400 South Olive AvenueWest...,"(400 S Olive Ave, West Palm Beach, FL 33401, U...","(26.7100477, -80.0517023, 0.0)",26.710048,-80.051702,0.0
2,134 and 142 Lakeview Avenue,Downtown West Palm Beach,20100000,NaN,NaN,NaN,277000,"Last year, Ross' Related paid $20.1 million fo...",https://therealdeal.com/miami/2022/10/04/ross-...,and https://therealdeal.com/miami/2021/11/12/s...,...,https://s12.therealdeal.com/trd/m/up/2022/10/M...,"Last year, Ross'",Related paid $20.1 million for a 2.5-acre pro...,"<a href=""https://therealdeal.com/miami/2022/10...",134 and 142 Lakeview AvenueDowntown West Palm ...,"(142 Lakeview Ave, West Palm Beach, FL 33401, ...","(26.7060308, -80.0503052, 0.0)",26.706031,-80.050305,0.0
3,515 Fern Street,Downtown West Palm Beach,NaN,NaN,NaN,NaN,NaN,"In September, Ross news broke that Ross planne...",https://therealdeal.com/miami/2022/09/16/steph...,NaN,...,https://s13.therealdeal.com/trd/m/up/2022/09/N...,"In September, Ross",news broke that Ross planned a 25-story tower...,"<a href=""https://therealdeal.com/miami/2022/09...",515 Fern StreetDowntown West Palm Beach,"(515 Fern St, West Palm Beach, FL 33401, USA, ...","(26.7107154, -80.0561871, 0.0)",26.710715,-80.056187,0.0
4,777 South Flagler Drive and 201 Lakeview Avenue,Downtown West Palm Beach,282000000,NaN,NaN,NaN,448914,"In January of 2021, he paid $282 million to an...",https://therealdeal.com/miami/2021/01/22/relat...,NaN,...,https://s12.therealdeal.com/trd/m/up/2021/01/1...,"In January of 2021,",he paid $282 million to an AEW Capital Manage...,"<a href=""https://therealdeal.com/miami/2021/01...",777 South Flagler Drive and 201 Lakeview Avenu...,"(201 Lakeview Ave, West Palm Beach, FL 33401, ...","(26.7063695, -80.0507588, 0.0)",26.706370,-80.050759,0.0
5,525 Okeechobee Boulevard,Downtown West Palm Beach,175000000,NaN,NaN,NaN,300000,"For the second time in 2021, Stephen Ross' Rel...",https://therealdeal.com/miami/2021/05/24/relat...,NaN,...,https://s11.therealdeal.com/trd/m/up/2021/05/1...,For the second time,"in 2021, Stephen Ross' Related shelled out ma...","<a href=""https://therealdeal.com/miami/2021/05...",525 Okeechobee BoulevardDowntown West Palm Beach,"(525 Okeechobee Blvd, West Palm Beach, FL 3340...","(26.7066126, -80.0559808, 0.0)",26.706613,-80.055981,0.0
6,222 Lakeview Avenue,Downtown West Palm Beach,unknown,,NaN,NaN,NaN,"Last year, Ross' Related bought half ownership...",https://therealdeal.com/miami/2021/06/11/relat...,NaN,...,https://s11.therealdeal.com/trd/m/up/2021/06/m...,"Last year, Ross'",Related bought half ownership interest in the...,"<a href=""https://therealdeal.com/miami/2021/06...",222 Lakeview AvenueDowntown West Palm Beach,"(222 Lakeview Ave, West Palm Beach, FL 33401, ...","(26.7056668, -80.0509601, 0.0)",26.705667,-80.050960,0.0
7,400 and 450 South Australian Avenue,Downtown West Palm Beach,35000000,NaN,NaN,NaN,NaN,"In February, R

## Correction section

In [32]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

### Change value to percent value
### df['percent_change'] = pd.Series(['{0:.2f}%'.format(val) for val in df['percent_change']], index = df.index)

## HTML popup formatter

In [33]:
df.columns

Index(['full_address', 'county', 'buy_price', 'sell_price', 'condos', 'apts',
       'office_sqft', 'description', 'story_link', 'additional_links',
       'other_notes', 'image_links', 'short_description', 'remaining_desc',
       'description_link', 'geo_address', 'loc', 'point', 'lat', 'lon',
       'altitude'],
      dtype='object')

In [34]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    image = df['image_links'].iloc[i]
    description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    <strong>{}'''.format(description) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [35]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [36]:
df.description_link[0]

'<a href="https://therealdeal.com/miami/2022/11/15/steve-ross-related-adds-broward-affordable-rentals-to-portfolio/" target="_blank" rel="noopener noreferrer">In November, Steve</a> Ross\' Related paid $55.5 million for 300 affordable apartments in Cooper City, Broward County. The sellers, ZOM Living and NRP Group, finished the Monterra community in 2012.'

In [37]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=8)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Stephen Ross\' Related Holdings in South Florida")

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [38]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [39]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [40]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Stephen_Ross_Related_SoFla
